# Quantum Protein Folding - User Guide

Welcome to the **Quantum Protein Folding** user manual. This guide will walk you through the process of simulating protein folding using quantum computing techniques. We will explore two approaches:

1.  **High-Level API**: Using the automated `setup_utils` for a quick start.
2.  **Low-Level API**: Manually initializing components for deeper understanding and customization.

## Prerequisites

Ensure you have the necessary dependencies installed. This project relies on `qiskit`, `numpy`, and other libraries defined in `pyproject.toml`.

> **Note**: This simulation is computationally intensive. For local execution, we recommend keeping protein chains short (Length $\le$ 5).

In [ ]:
import sys
import os
from pathlib import Path
import logging

# Add the src directory to the system path to import project modules
current_dir = Path(os.getcwd())
project_root = current_dir.parent
src_path = project_root / "src"

if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Configure logging
from logger import get_logger
logger = get_logger()
logger.setLevel(logging.INFO)

# Part 1: High-Level API (Automated Workflow)

The `utils.setup_utils` module provides a streamlined interface to run the simulation. This is the recommended way for standard use cases.

## 1. Define the Protein Sequence

We start by defining the main chain of the protein. We also define a side chain, which is currently a placeholder in this model.

In [ ]:
from constants import EMPTY_SIDECHAIN_PLACEHOLDER

# Define a short protein sequence
main_chain = "APRLR"
side_chain = EMPTY_SIDECHAIN_PLACEHOLDER * len(main_chain)

print(f"Main Chain: {main_chain}")
print(f"Side Chain: {side_chain}")

## 2. Setup the Folding System

The `setup_folding_system` function initializes the core components:
- **Protein**: Represents the biological structure.
- **Interaction**: Defines the physical forces (e.g., Miyazawa-Jernigan or Hydrophobic-Polar).
- **ContactMap**: Pre-calculates possible contacts between beads.
- **DistanceMap**: Pre-calculates distances on the grid.

In [ ]:
from utils.setup_utils import setup_folding_system

protein, interaction, contact_map, distance_map = setup_folding_system(
    main_chain=main_chain, 
    side_chain=side_chain
)

print("System initialized successfully.")

## 3. Build and Compress Hamiltonian

We construct the Hamiltonian, which encodes the energy landscape of the protein folding problem. We then compress it to remove unused qubits, optimizing it for the quantum solver.

In [ ]:
from utils.setup_utils import build_and_compress_hamiltonian

original_h, compressed_h = build_and_compress_hamiltonian(
    protein=protein,
    interaction=interaction,
    contact_map=contact_map,
    distance_map=distance_map,
)

print(f"Original Qubits: {original_h.num_qubits}")
print(f"Compressed Qubits: {compressed_h.num_qubits}")

## 4. Run VQE Optimization

We use the Variational Quantum Eigensolver (VQE) to find the minimum energy configuration (the folded state).

In [ ]:
from utils.setup_utils import setup_vqe_optimization, run_vqe_optimization

# Setup VQE with the required number of qubits
vqe, counts, values = setup_vqe_optimization(num_qubits=compressed_h.num_qubits)

# Run the optimization
raw_results = run_vqe_optimization(vqe=vqe, hamiltonian=compressed_h)

print(f"Optimization Complete. Minimum Eigenvalue: {raw_results.eigenvalue}")

## 5. Analyze and Visualize Results

Finally, we interpret the quantum results to reconstruct the 3D structure of the protein.

In [ ]:
from utils.setup_utils import setup_result_analysis

result_interpreter, result_visualizer = setup_result_analysis(
    raw_results=raw_results,
    protein=protein,
    vqe_iterations=counts,
    vqe_energies=values,
)

# Generate visualizations
result_visualizer.visualize_3d()
print(f"Results saved to: {result_visualizer.dirpath}")

---
# Part 2: Low-Level API (Manual Workflow)

For advanced users, you can manually instantiate each component. This allows for greater control over the interaction models and Hamiltonian construction.

## 1. Manual Initialization

Here we explicitly choose the `MJInteraction` model and create the `Protein` object.

In [ ]:
from protein import Protein
from interaction import MJInteraction
from contact import ContactMap
from distance import DistanceMap

# Initialize Interaction Model
interaction = MJInteraction()

# Initialize Protein
protein = Protein(
    main_protein_sequence=main_chain,
    side_protein_sequence=side_chain,
    valid_symbols=interaction.valid_symbols
)

# Calculate Maps
contact_map = ContactMap(protein=protein)
distance_map = DistanceMap(protein=protein)

## 2. Manual Hamiltonian Construction

We use the `HamiltonianBuilder` to sum up the energy terms.

In [ ]:
from builder import HamiltonianBuilder
from utils.qubit_utils import remove_unused_qubits

h_builder = HamiltonianBuilder(
    protein=protein,
    interaction=interaction,
    distance_map=distance_map,
    contact_map=contact_map,
)

full_hamiltonian = h_builder.sum_hamiltonians()
compressed_hamiltonian = remove_unused_qubits(full_hamiltonian)

print(f"Manually built Hamiltonian with {compressed_hamiltonian.num_qubits} qubits.")

## 3. Manual VQE Setup

We can configure the VQE solver with specific optimizers and ansatzes using Qiskit directly.

In [ ]:
from qiskit_algorithms import SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import RealAmplitudes
from backend import get_sampler

# Setup Ansatz
ansatz = RealAmplitudes(num_qubits=compressed_hamiltonian.num_qubits, reps=1)

# Setup Optimizer
optimizer = COBYLA(maxiter=50)

# Get Sampler (Backend)
sampler, _ = get_sampler()

# Initialize VQE
vqe = SamplingVQE(
    sampler=sampler,
    ansatz=ansatz,
    optimizer=optimizer,
    aggregation=0.1
)

# Run
result = vqe.compute_minimum_eigenvalue(compressed_hamiltonian)
print(f"Manual VQE Result: {result.eigenvalue}")